In [92]:
import os
import guidance
import requests 
import openai
import json
from enum import Enum
from dotenv import load_dotenv
from duffel_api import Duffel
from duffel_api.http_client import Pagination
from typing import Optional

load_dotenv("../.env")
duffel = Duffel(access_token=os.getenv("DUFFEL_API_KEY"))
openai_api_key = os.getenv("OPENAI_API_KEY")

In [99]:
class DuffelManager:
    def __init__(self, api_client):
        self.api_client = api_client

    def call_duffel_api(self, departure_city: str, destination_city: str, departure_date: str, cabin_class: str = None) -> str:
        slices = [{"origin": departure_city, "destination": destination_city, "departure_date": departure_date}]

        try:
            offer_request = self.api_client.offer_requests.create().slices(slices).passengers([{"type": "adult"}])
            if cabin_class:
                offer_request.cabin_class(cabin_class)
            offer_request_response = offer_request.execute()
            return offer_request_response.id
        except Exception as e:
            # Need better error handling and logging
            raise

    @staticmethod
    def parse_offers(offers) -> list:
        return [
            {
                "id": offer.id,
                "total_amount": offer.total_amount,
                "airline_code": offer.owner.iata_code,
                "airline_name": offer.owner.name,
                "num_segments": len(offer.slices), 
                "departure_time": offer.slices[0].segments[0].departing_at,
                "arrival_time": offer.slices[-1].segments[-1].arriving_at,
            }
            for offer in offers
        ]
    
    @staticmethod
    def calculate_offer_score(offer, cost_weight, segments_weight, min_cost, max_segments):
        def normalize_value(value, min_value, max_value):
            if max_value - min_value == 0: return 0
            return (value - min_value) / (max_value - min_value)
        
        normalized_cost = normalize_value(float(offer["total_amount"]), min_cost, min_cost)
        normalized_segments = normalize_value(offer["num_segments"], 0, max_segments)
        return (cost_weight * normalized_cost) + (segments_weight * normalized_segments)
    
    def get_best_offer(self, offers: list, airline: Optional[str] = None) -> Optional[dict]:
        cost_weight = 0.7
        segments_weight = 0.3
    
        airline_filtered = []
        if airline is not None:
            airline_filtered = [offer for offer in offers if offer["airline_code"].upper() == airline.upper()]
            print(f"Num airlines after filtering {len(airline_filtered)}")
    
        filtered_offers = offers if len(airline_filtered) == 0 else airline_filtered
        min_cost = min(float(offer["total_amount"]) for offer in filtered_offers)
        max_segments = max(offer["num_segments"] for offer in filtered_offers)
    
        best_offer = min(filtered_offers, key=lambda offer: self.calculate_offer_score(offer, cost_weight, segments_weight, min_cost, max_segments))
        return best_offer

    def get_offer(self, departure_city: str, destination_city: str, departure_date: str, airline: str = None, cabin_class: str = None) -> dict:
        try:
            offer_request_id = self.call_duffel_api(departure_city, destination_city, departure_date, cabin_class)
            offers_object = self.api_client.offers.list(offer_request_id, sort="total_amount")
            offers = self.parse_offers(offers_object)
            return self.get_best_offer(offers, airline=airline)
        except Exception as e: # Need better error handling and logging
            raise  


In [100]:
caller = DuffelManager(duffel)

In [101]:
caller.get_offer("NYC", "ATL", "2023-12-11", airline="DL")

0


{'id': 'off_0000AbdfOPo6nEW20UCoUT',
 'total_amount': '62.89',
 'airline_code': 'NK',
 'airline_name': 'Spirit Airlines',
 'num_segments': 1,
 'departure_time': datetime.datetime(2023, 12, 11, 22, 10),
 'arrival_time': datetime.datetime(2023, 12, 12, 0, 44)}

In [85]:
test.slices[0].segments[0].departing_at

datetime.datetime(2023, 12, 21, 15, 29)